In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

First question: Which genre of track sells the most tracks? Based on their genres, which albums should the record company choose to add to their collection?

In [13]:
%%sql
SELECT name, type
FROM sqlite_master
WHERE type IN ("table", "view")

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [11]:
%%sql
SELECT *
FROM employee
WHERE title= "IT Staff"


Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2017-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2017-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


In [52]:
%%sql

WITH c_tn_g AS
(
SELECT i.billing_country, t.name track_name, g.name genre_name
FROM invoice i
INNER JOIN invoice_line il ON i.invoice_id=il.invoice_id
INNER JOIN track t ON il.track_id=t.track_id
LEFT JOIN genre g ON t.genre_id = g.genre_id 
WHERE billing_country = "USA"
)

SELECT genre_name, COUNT(track_name) number_tracks_sold, 
                                                         (CAST(COUNT(track_name) AS float)/(SELECT COUNT(track_name) 
                                                         FROM c_tn_g percentage))*100 percentage_of_tracks_sold
FROM c_tn_g
GROUP BY genre_name
ORDER BY percentage_of_tracks_sold DESC



Done.


genre_name,number_tracks_sold,percentage_of_tracks_sold
Rock,561,53.37773549000951
Alternative & Punk,130,12.369172216936251
Metal,124,11.798287345385347
R&B/Soul,53,5.042816365366318
Blues,36,3.425309229305423
Alternative,35,3.3301617507136063
Latin,22,2.093244529019981
Pop,22,2.093244529019981
Hip Hop/Rap,20,1.9029495718363463
Jazz,14,1.3320647002854424


Rock is the genre with the highest number of tracks sold. 53.37% of tracks sold are from this genre! But Rock is not one of our 4 options. Punk is the next most popular, so we should certainly chose "Red Tone" as one of our 3 albums (although we should be careful about this because "alternative" is also listed under this genre, so we can't be completely sure Punk alone is high performing). Blues comes next in the list (3.42%) so we should choose Slim Jim Bites, followed by Pop (2.09%) so we should choose Meteor and the Girls. We should not choose Regal, becuse Hip Hop is the lowest performing genre of our options.

Second question: What is the total dollar amount of sales for each employee within the company?

In [63]:
%%sql
SELECT e.first_name||" "||e.last_name employee_name, SUM(i.total) sales_total, e.hire_date                                                      
FROM employee e
INNER JOIN customer c ON e.employee_id=c.support_rep_id
INNER JOIN invoice i ON c.customer_id=i.customer_id
GROUP BY employee_name

Done.


employee_name,sales_total,hire_date
Jane Peacock,1731.5100000000039,2017-04-01 00:00:00
Margaret Park,1584.0000000000034,2017-05-03 00:00:00
Steve Johnson,1393.920000000002,2017-10-17 00:00:00


Jane has made more sales that Margaret or Steve. But she was hired before the other two, so that could account for the difference. We shouldn't immediately think she's a better employee!

In [75]:
%%sql
WITH country_or_other AS
(SELECT i.invoice_id, CASE
                   WHEN (SELECT COUNT(*) 
                   FROM customer
                   WHERE country=c.country) =1 THEN "Other"
                   ELSE c.country
                END AS country, c.customer_id, i.total
FROM customer c
INNER JOIN invoice i ON c.customer_id=i.customer_id
)

SELECT country,
       COUNT(DISTINCT customer_id) number_customers, 
       SUM(total) total_sales,
       SUM(total)/COUNT(DISTINCT customer_id) avg_sales_per_customer,
       SUM(total)/COUNT(invoice_id) avg_order_value,
       CASE
          WHEN country= "Other" THEN 1
          ELSE 0
          END AS sort  
FROM country_or_other
GROUP BY country
ORDER BY sort ASC, total_sales DESC

         
            

Done.


country,number_customers,total_sales,avg_sales_per_customer,avg_order_value,sort
USA,13,1040.4899999999998,80.0376923076923,7.942671755725189,0
Canada,8,535.5900000000001,66.94875000000002,7.047236842105265,0
Brazil,5,427.68000000000006,85.53600000000002,7.0111475409836075,0
France,5,389.0699999999999,77.81399999999998,7.781399999999998,0
Germany,4,334.62,83.655,8.161463414634147,0
Czech Republic,2,273.24000000000007,136.62000000000003,9.108000000000002,0
United Kingdom,3,245.52,81.84,8.768571428571429,0
Portugal,2,185.13000000000002,92.56500000000001,6.383793103448276,0
India,2,183.14999999999998,91.57499999999999,8.72142857142857,0
Other,15,1094.9400000000003,72.99600000000002,7.44857142857143,1


Question 3: Should the record company stop buying full albums? What percentage of invoices were purchasing an entire album?

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [24]:
%%sql
WITH invoice_first_track AS
    (SELECT il.invoice_id invoice_id, MIN(il.track_id) first_track_id
    FROM invoice_line il
    GROUP BY 1
    )
    
SELECT ift.*,
    CASE
        WHEN (SELECT t.track_id FROM track t
              WHERE t.album_id = (SELECT t2.album_id FROM track t2
                                  WHERE t2.track_id=ift.first_track_id)
              EXCEPT 
              
              SELECT il2.track_id FROM invoice_line il2
              WHERE il2.invoice_id= ift.invoice_id
             ) IS NULL
        
             AND
            
            (SELECT il2.track_id FROM invoice_line il2
              WHERE il2.invoice_id= ift.invoice_id
                
            EXCEPT
            
            SELECT t.track_id FROM track t
              WHERE t.album_id = (SELECT t2.album_id FROM track t2
                                  WHERE t2.track_id=ift.first_track_id)
            )
             IS NULL
            
        THEN "yes"
        ELSE "no"
    END AS "album_purchase"

FROM invoice_first_track ift
GROUP BY album_purchase
    

Done.


invoice_id,first_track_id,album_purchase
614,2650,no
612,2204,yes


In [3]:
%%sql
WITH invoice_first_track AS
    (SELECT il.invoice_id invoice_id, MIN(il.track_id) first_track_id
    FROM invoice_line il
    GROUP BY 1
    )
    
SELECT 
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice) percent_of_invoices
    
FROM 
(
SELECT ift.*,
    CASE
        WHEN (SELECT t.track_id FROM track t
              WHERE t.album_id = (SELECT t2.album_id FROM track t2
                                  WHERE t2.track_id=ift.first_track_id)
              EXCEPT 
              
              SELECT il2.track_id FROM invoice_line il2
              WHERE il2.invoice_id= ift.invoice_id
             ) IS NULL
        
             AND
            
            (SELECT il2.track_id FROM invoice_line il2
              WHERE il2.invoice_id= ift.invoice_id
                
            EXCEPT
            
            SELECT t.track_id FROM track t
              WHERE t.album_id = (SELECT t2.album_id FROM track t2
                                  WHERE t2.track_id=ift.first_track_id)
            )
             IS NULL
            
        THEN "yes"
        ELSE "no"
    END AS "album_purchase"

FROM invoice_first_track ift
)
GROUP BY album_purchase
    

Done.


album_purchase,number_of_invoices,percent_of_invoices
no,500,0.8143322475570033
yes,114,0.18566775244299674


18.56 % of purchases are album purchases, so I would recommend eliminating albums from the store only if the savings potential is higher than 18.56% of revenue.